<a href="https://colab.research.google.com/github/ErnestoCash/Ind-4.0/blob/main/predictor_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar yfinance si no está instalado
!pip install yfinance

# Importar bibliotecas necesarias
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

In [ ]:
# Función para obtener datos históricos de precios de acciones
def obtener_datos(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

# Definir el símbolo de la acción y el rango de fechas
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2021-01-01'

# Obtener datos históricos de precios de acciones
stock_data = obtener_datos(symbol, start_date, end_date)

In [ ]:
# Preprocesamiento de datos
scaler = MinMaxScaler(feature_range=(0,1))  # Escalador MinMax para escalar los datos entre 0 y 1
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1,1))  # Escalar los datos de precios de cierre

# Dividir datos en entrenamiento y prueba
train_data, test_data = train_test_split(scaled_data, test_size=0.2, shuffle=False)  # Dividir datos en 80% entrenamiento y 20% prueba

In [ ]:
# Preparar datos para modelo LSTM
def preparar_datos(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps - 1):
        X.append(data[i:(i + time_steps), 0])  # Crear secuencias de datos para características (X)
        y.append(data[i + time_steps, 0])      # Crear secuencias de datos para la variable objetivo (y)
    return np.array(X), np.array(y)

In [ ]:
time_steps = 60  # Número de pasos de tiempo para considerar en cada secuencia de datos
X_train, y_train = preparar_datos(train_data, time_steps)  # Preparar datos de entrenamiento
X_test, y_test = preparar_datos(test_data, time_steps)     # Preparar datos de prueba

In [ ]:
# Reshape de datos para modelo LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Agregar dimensión para LSTM (muestras, pasos de tiempo, características)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Crear modelo LSTM
model = Sequential()  # Inicializar modelo secuencial
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # Capa LSTM con 50 unidades, return_sequences=True para retornar secuencias para capas posteriores, input_shape para especificar forma de entrada
model.add(LSTM(units=50, return_sequences=False))  # Capa LSTM con 50 unidades, return_sequences=False para no retornar secuencias para capas posteriores
model.add(Dense(units=25))  # Capa densa con 25 neuronas
model.add(Dense(units=1))   # Capa de salida con 1 neurona para la predicción del precio

In [ ]:
# Compilar modelo
model.compile(optimizer='adam', loss='mean_squared_error')  # Compilar modelo con optimizador Adam y función de pérdida de error cuadrático medio

In [ ]:
# Entrenar modelo
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Entrenar modelo con 10 épocas y un tamaño de lote de 32

In [ ]:
# Obtener predicciones
predictions = model.predict(X_test)  # Realizar predicciones con datos de prueba
predictions = scaler.inverse_transform(predictions)  # Desescalar las predicciones

In [ ]:
# Visualizar resultados
plt.figure(figsize=(10,6))
plt.plot(stock_data.index[-len(predictions):], predictions, color='red', label='Predicted Stock Price')  # Graficar precios predichos
plt.plot(stock_data.index[-len(predictions):], scaler.inverse_transform(test_data[-len(predictions):]), color='blue', label='Actual Stock Price')  # Graficar precios reales
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
